In [1]:
ls

drive/  sample_data/


In [2]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 18.1 MB/s eta 0:00:00


In [4]:
import os
from os import mkdir
from torch import nn, optim
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from data import *
from utils import *
from Net import *
import numpy as np
import sys

In [5]:
if torch.cuda.is_available():
    device = 'cuda'
    print("Using cuda")
else:
    device = 'cpu'
    print("Using CPU")

Using cuda


In [19]:
weightPath = r'../2/params/best_model_epoch_5.pth'
filePath = r'../2/imgs'

In [14]:
dataPath= r'../2'
validationDataset = ValidationImageDataSet(dataPath)

In [15]:
for i in range(len(validationDataset)):
    print(validationDataset.name[i])

s013.nii
s015.nii


In [16]:
# import shutil
# copy_dir = r'../2/testimgs'
# todir = r'../2/imgs'
# for filename in validationDataset.name:
#     # 创建源文件和目标文件的完整路径
#     src_file = os.path.join(copy_dir, filename)
#     dst_file = os.path.join(todir, filename)

#     # 复制文件
#     shutil.copy(src_file, dst_file)

In [13]:
    imagePath = dataPath + '/testimgs'
    maskPath = dataPath + '/testmasks'
    resultPath = dataPath + '/results'

In [9]:
net = UNet().to(device)

In [10]:
batchSize = 1

In [17]:
    if os.path.exists(weightPath):
        #net.load_state_dict(torch.load(weightPath, map_location=torch.device(device)))
        net.load_state_dict(torch.load(weightPath, map_location = 'cpu'))
        print("Loading Weight Successful")
    else:
        print("Loading Weight Failed")

Loading Weight Successful


In [20]:
    for i in range(len(validationDataset)):
        image = validationDataset[i]
        # print(image)
        image = torch.unsqueeze(image, 0)
        image = image.to(device)
        with torch.no_grad():
            outImage = net(image)
        result = outImage.clone().detach().cpu().numpy()
        result = result.squeeze(0).squeeze(0)
        # mask_resized = itpl.zoom(result, (152 / 160, 72 / 80, 152 / 160), mode='constant')
        mask_ = result.transpose([2, 0, 1])
        origin_nii = sitk.ReadImage(filePath + '/' + validationDataset.name[i])#r'D:\0a\brainExtraction\data\source\subj011.nii')
        origin = origin_nii.GetOrigin()
        direction = origin_nii.GetDirection()
        space = origin_nii.GetSpacing()
        origin_arr = sitk.GetArrayFromImage(origin_nii)
        # print(np.shape(origin_arr))
        savedImg = sitk.GetImageFromArray(mask_)
        savedImg.SetOrigin(origin)
        savedImg.SetDirection(direction)
        savedImg.SetSpacing(space)
        sitk.WriteImage(savedImg, resultPath + '/' + validationDataset.name[i][0:validationDataset.name[i].find('.')] + '_mask.nii')#r'D:\0a\brainExtraction\data\results\subj011_mask.nii')
    print('Brain Extraction Done')

Brain Extraction Done
